References:
https://github.com/huggingface/notebooks/blob/master/sagemaker/08_distributed_summarization_bart_t5/sagemaker-notebook.ipynb

In [1]:
!pip install sagemaker transformers --upgrade

     |████████████████████████████████| 447 kB 30.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 59.8 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.63.0-py2.py3-none-any.whl size=627084 sha256=4446e0f2c199f0156a9e4ed01a0a19873fe71e62ff695612cb3c9a3c01e86aa9
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d9/2b/d5/1512cb99b7cf9958286688047ee6a37f7c23aa4faea9eb4487
Successfully built sagemaker
  Attempting uninstall: transformers
    Found existing installation: transformers 4.10.2
    Uninstalling transformers-4.10.2:
      Successfully uninstalled transformers-4.10.2
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.59.3
    Uninstalling sagemaker-2.59.3:
      Successfully uninstalled sagemaker-2.59.3
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

IAM role arn used for running training: arn:aws:iam::727118255515:role/service-role/AmazonSageMaker-ExecutionRole-20211011T142050
S3 bucket used for storing artifacts: sagemaker-eu-west-1-727118255515


In [3]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'}

## Configure distributed training and hyperparameters


In [16]:
hyperparameters={'per_device_train_batch_size': 4,
                 'per_device_eval_batch_size': 4,
                 'model_name_or_path': 'facebook/bart-large-cnn',
                 'do_train': True,
                 'do_eval': True,
                 'do_predict': True,
                 'train_file': '/home/ec2-user/SageMaker/data_train.json',
                 'validation_file': '/home/ec2-user/SageMaker/data_val.json',
                 'test_file': '/home/ec2-user/SageMaker/data_test.json',
                 'text_column': 'text',
                 'summary_column': 'summary',
                 'predict_with_generate': True,
                 'output_dir': '/opt/ml/model',
                 'overwrite_output_dir': True,
                 'num_train_epochs': 3,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': True,
                 }

# configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

## Create a HuggingFace estimator and start training

In [17]:
from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point='run_summarization.py', # script
    source_dir='./examples/pytorch/summarization', # relative path to example
    git_config=git_config,
    instance_type='local_gpu',
    instance_count=1,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
    role=role,
    hyperparameters = hyperparameters,
    # distribution = distribution
)

In [18]:
# starting the train job
huggingface_estimator.fit()

Creating 76ofjdgaod-algo-1-nhrkk ... 
Creating 76ofjdgaod-algo-1-nhrkk ... done
Attaching to 76ofjdgaod-algo-1-nhrkk
76ofjdgaod-algo-1-nhrkk | 2021-10-13 19:19:19,300 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
76ofjdgaod-algo-1-nhrkk | 2021-10-13 19:19:19,325 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
76ofjdgaod-algo-1-nhrkk | 2021-10-13 19:19:19,328 sagemaker_pytorch_container.training INFO     Invoking user training script.
76ofjdgaod-algo-1-nhrkk | 2021-10-13 19:19:19,478 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
76ofjdgaod-algo-1-nhrkk | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
76ofjdgaod-algo-1-nhrkk | Requirement already satisfied: datasets>=1.1.3 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (1.6.2)
76ofjdgaod-algo-1-nhrkk | Requirement already satisfied: sentencepiece!=0.1.92 in /opt/conda/lib/p

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpg9yx5oya/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1